In [129]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss

from crepes import ConformalPredictiveSystem
from crepes.extras import binning, DifficultyEstimator

from calibrated_explanations import  VennAbers, __version__
from calibrated_explanations.utils import transform_to_numeric
from time import time

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

print(f"calibrated_explanations {__version__}")



calibrated_explanations v0.3.1


In [131]:

datasets = {'COMPAS': 'classification', 'Adult': 'classification', 'German': 'classification', 'Housing': 'regression', 'CaC': 'regression'}
results = {}
for name, type in datasets.items():
    results[name] = {}
    results[name]['type'] = type
    for category in ['mondrian', 'mondrian_x', 'model_split']:
        results[name][category] = {'time': 0, 'base': 0, 'rand': 0, 'race': 0, 'sex': 0, 'age': 0, 'edu': 0, 'crm': 0}

num_rep = 1
kf = KFold(n_splits=10)

In [132]:
def coverage(y_true, low, up):
    return np.mean((y_true >= low) & (y_true <= up))

# COMPAS
Data preprocessing from [this notebook](https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_COMPAS_with_SHAP.ipynb#scrollTo=KF00pJvkeicT).

In [133]:
dataset = 'COMPAS'
df = pd.read_csv('https://storage.googleapis.com/what-if-tool-resources/computefest2019/cox-violent-parsed_filt.csv')
print(df.shape)
# Preprocess the data

# Filter out entries with no indication of recidivism or no compass score
df = df[df['is_recid'] != -1]
df = df[df['decile_score'] != -1]


# Make the COMPASS label column numeric (0 and 1), for use in our model
df['score_text'] = np.where(df['score_text'] == 'Low', 'Low', 'Not Low')

target = 'score_text'

(18316, 40)


In [134]:
features_to_keep = ['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'score_text']
# 'age_cat', 'is_recid', 'vr_charge_desc',  'c_charge_desc',, 'is_violent_recid''vr_charge_degree', 'c_charge_degree', 'decile_score', 
df = df[features_to_keep]
df, categorical_features, categorical_labels, target_labels, mappings = transform_to_numeric(df, target)


In [135]:
print(df.columns)
race = 2
age = 1
sex = 0

Index(['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'score_text'],
      dtype='object')


In [136]:
num_to_test = 20 # number of instances to test

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop([target],axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test

X, y = Xd.values, yd.values

In [137]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':race, 'sex':sex, 'age':age}
attribute_names = [base, rand, 'race', 'sex', 'age']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestClassifier()
        model.fit(x_train,y_train)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():                
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results[dataset]['mondrian_x']['time'] = time() - time_mondrian_x
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [138]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, age]
attribute_names = [base, rand, 'race', 'sex', 'age']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()

        model.fit(X_train,y_train)
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [139]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, age]
attribute_names = [base, rand, 'race', 'sex', 'age']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []
    
time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_train_bin = binning(X_train[:,age], age_cal_boundaries)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': X_train[:,race], 'sex': X_train[:,sex], 'age': age_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestClassifier()                
                model.fit(X_train_bin,y_train_bin)
                uncal = model.predict_proba(X_test_bin)
                va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                model_split['uncal'][attr].append(uncal[:,1])
                model_split['proba'][attr].append(proba[:,1])
                model_split['low'][attr].append(low)
                model_split['high'][attr].append(high)
                model_split['width'][attr].append(high - low)
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [140]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [141]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.0079 	 0.0072 	 0.0071
rand width: 	 0.0231 	 0.0219 	 0.0198
race width: 	 0.0192 	 0.0189 	 0.0185
sex width: 	 0.0122 	 0.0112 	 0.0105
age width: 	 0.0249 	 0.0215 	 0.0205


In [142]:
print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}', end='')
    print(f' \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .4f}', end='')
    print(f' \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
    

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base Accuracy: 	 0.7546 	 0.7742 	 0.7738 	 0.7558 	 0.7764 	 0.7720
rand Accuracy: 	 0.7499 	 0.7738 	 0.7292 	 0.7558 	 0.7764 	 0.7276
race Accuracy: 	 0.7579 	 0.7754 	 0.7720 	 0.7558 	 0.7764 	 0.7702
sex Accuracy: 	 0.7559 	 0.7743 	 0.7737 	 0.7558 	 0.7764 	 0.7708
age Accuracy: 	 0.7522 	 0.7749 	 0.7706 	 0.7558 	 0.7764 	 0.7704

base LogLoss: 	 0.4960 	 0.4642 	 0.4674 	 0.6682 	 0.7861 	 0.7886
rand LogLoss: 	 0.5010 	 0.4685 	 0.5420 	 0.6682 	 0.7861 	 0.9364
race LogLoss: 	 0.4860 	 0.4631 	 0.4691 	 0.6682 	 0.7861 	 0.8425
sex LogLoss: 	 0.4952 	 0.4639 	 0.4665 	 0.6682 	 0.7861 	 0.7958
age LogLoss: 	 0.4979 	 0.4634 	 0.4713 	 0.6682 	 0.7861 	 0.7943


In [143]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


# Adult

In [144]:
dataset = 'Adult'
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [145]:
df = X
target = 'Income'
y = y.replace('<=50K.', '<=50K')
y = y.replace('>50K.', '>50K')
df[target] = y
df = df.dropna()
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)
print(target_labels)

{0: '<=50K', 1: '>50K'}


In [146]:
num_to_test = 10 # number of instances to test, one from each class

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop(target,axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test
X, y = Xd.values, yd.values

In [147]:
print(df.columns)
edu = 3
race = 8
sex = 9

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Income'],
      dtype='object')


In [148]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':race, 'sex':sex, 'edu':edu}
attribute_names = [base, rand, 'race', 'sex', 'edu']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestClassifier()
        model.fit(x_train,y_train)
        
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():            
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results[dataset]['mondrian_x']['time'] = time() - time_mondrian_x
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [149]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, edu]
attribute_names = [base, rand, 'race', 'sex', 'edu']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [150]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, edu]
attribute_names = [base, rand, 'race', 'sex', 'edu']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        

        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': X_train[:,race], 'sex': X_train[:,sex], 'edu': X_train[:,edu]}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        try:
            for attr in attribute_names:
                for bin in np.unique(train_bins[attr]):
                    X_train_bin = X_train[train_bins[attr] == bin]
                    y_train_bin = y_train[train_bins[attr] == bin]
                    X_cal_bin = X_cal[cal_bins[attr] == bin]
                    y_cal_bin = y_cal[cal_bins[attr] == bin]
                    X_test_bin = X_test[test_bins[attr] == bin]
                    y_test_bin = y_test[test_bins[attr] == bin]
                
                    if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                        continue
                    
                    model = RandomForestClassifier()
                    model.fit(X_train_bin,y_train_bin)
                    uncal = model.predict_proba(X_test_bin)
                    va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                    proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                    model_split['uncal'][attr].append(uncal[:,1])
                    model_split['proba'][attr].append(proba[:,1])
                    model_split['low'][attr].append(low)
                    model_split['high'][attr].append(high)
                    model_split['width'][attr].append(high - low)
                    model_split['y'][attr].append(y_test_bin)
        except Exception as e:
            print(e)
            print(f'attr: {attr} bin: {bin}')    

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14


In [151]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [152]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.0028 	 0.0029 	 0.0030
rand width: 	 0.0091 	 0.0096 	 0.0098
race width: 	 0.0064 	 0.0068 	 0.0066
sex width: 	 0.0045 	 0.0046 	 0.0046
edu width: 	 0.0153 	 0.0159 	 0.0150


In [153]:
print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}', end='')
    print(f' \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .4f}', end='')
    print(f' \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
    

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base Accuracy: 	 0.8315 	 0.8552 	 0.8547 	 0.8322 	 0.8558 	 0.8545
rand Accuracy: 	 0.8308 	 0.8539 	 0.8483 	 0.8322 	 0.8558 	 0.8490
race Accuracy: 	 0.8311 	 0.8554 	 0.8517 	 0.8322 	 0.8558 	 0.8512
sex Accuracy: 	 0.8309 	 0.8553 	 0.8533 	 0.8322 	 0.8558 	 0.8534
edu Accuracy: 	 0.8333 	 0.8543 	 0.8495 	 0.8322 	 0.8558 	 0.8487

base LogLoss: 	 0.3612 	 0.3186 	 0.3180 	 0.4158 	 0.3705 	 0.3670
rand LogLoss: 	 0.3628 	 0.3203 	 0.3306 	 0.4158 	 0.3705 	 0.3652
race LogLoss: 	 0.3616 	 0.3188 	 0.3225 	 0.4158 	 0.3705 	 0.3605
sex LogLoss: 	 0.3607 	 0.3179 	 0.3210 	 0.4158 	 0.3705 	 0.3730
edu LogLoss: 	 0.3600 	 0.3199 	 0.3313 	 0.4158 	 0.3705 	 0.3901


In [154]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


# German


In [155]:
dataset = 'German'
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
# data (as pandas dataframes) 
X = statlog_german_credit_data.data.features 
y = statlog_german_credit_data.data.targets 

In [156]:
feature_names = statlog_german_credit_data.variables.description[:-1]
feature_names[0] = 'Status'
feature_names[7] = 'Installment rate'
feature_names[15] = 'Existing credits'
feature_names[17] = 'Num people liable'
target_labels = {1: 'Good', 0: 'Bad'}

In [157]:
print(feature_names)
age = 12
sex = 8

0                         Status
1                       Duration
2                 Credit history
3                        Purpose
4                  Credit amount
5          Savings account/bonds
6       Present employment since
7               Installment rate
8        Personal status and sex
9     Other debtors / guarantors
10       Present residence since
11                      Property
12                           Age
13       Other installment plans
14                       Housing
15              Existing credits
16                           Job
17             Num people liable
18                     Telephone
19                foreign worker
Name: description, dtype: object


In [158]:
df = X
target = 'Class'
df[target] = y
df = df.dropna()
df, categorical_features, categorical_labels, _, _ = transform_to_numeric(df, target)

In [159]:
num_to_test = 10 # number of instances to test, one from each class

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop(target,axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test
X, y = Xd.values, yd.values
y[y == 2] = 0

In [160]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'sex':sex, 'age':age}
attribute_names = [base, rand, 'sex', 'age']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestClassifier()
        model.fit(x_train,y_train)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results[dataset]['mondrian_x']['time'] = time() - time_mondrian_x
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [161]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, sex, age]
attribute_names = [base, rand, 'sex', 'age']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [162]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, sex, age]
attribute_names = [base, rand, 'sex', 'age']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_train_bin = binning(X_train[:,age], age_cal_boundaries)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'sex': X_train[:,sex], 'age': age_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestClassifier()                
                model.fit(X_train_bin,y_train_bin)
                uncal = model.predict_proba(X_test_bin)
                va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                model_split['uncal'][attr].append(uncal[:,1])
                model_split['proba'][attr].append(proba[:,1])
                model_split['low'][attr].append(low)
                model_split['high'][attr].append(high)
                model_split['width'][attr].append(high - low)
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [163]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [164]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.0369 	 0.0467 	 0.0419
rand width: 	 0.1191 	 0.1350 	 0.1181
sex width: 	 0.0962 	 0.1008 	 0.0892
age width: 	 0.1198 	 0.1381 	 0.1255


In [165]:
print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}', end='')
    print(f' \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .4f}', end='')
    print(f' \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
    

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base Accuracy: 	 0.7730 	 0.7480 	 0.7390 	 0.7680 	 0.7460 	 0.7520
rand Accuracy: 	 0.7620 	 0.7540 	 0.7070 	 0.7680 	 0.7460 	 0.7120
sex Accuracy: 	 0.7710 	 0.7370 	 0.7200 	 0.7680 	 0.7460 	 0.7390
age Accuracy: 	 0.7590 	 0.7660 	 0.7130 	 0.7680 	 0.7460 	 0.7340

base LogLoss: 	 0.5089 	 0.4988 	 0.5182 	 0.5111 	 0.5037 	 0.5136
rand LogLoss: 	 0.5164 	 0.5065 	 0.5572 	 0.5111 	 0.5037 	 0.5483
sex LogLoss: 	 0.5151 	 0.5023 	 0.5435 	 0.5111 	 0.5037 	 0.5366
age LogLoss: 	 0.5242 	 0.5004 	 0.5505 	 0.5111 	 0.5037 	 0.5374


In [166]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


# Boston

In [167]:
dataset = 'Housing'
df = pd.read_csv('../data/reg/HousingData.csv', na_values='NA')
# df.head()

In [168]:
feature_names = df.columns[:-1]
print(feature_names)
crm = 0

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')


In [169]:
target = 'MEDV'
df = df.dropna()
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)

In [170]:
num_to_test = 10 # number of instances to test, one from each class

Xd, yd = df.drop(target,axis=1), df[target]
X, y = df.drop(target,axis=1).values, df[target].values
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

### Standard Regression
Default confidence of 90%

In [171]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'crm':crm}
attribute_names = [base, rand, 'crm']
cps = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['median'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['median'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestRegressor()
        model.fit(x_train,y_train)
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=3)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'crm': crm_test_bin}
        residuals_cal = model.predict(x_cal) - y_cal
        y_test_hat = model.predict(x_test)
        mondrian_x['y'].append(y_test)        
        
        for attr, a in attributes.items():                
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian_x['uncal'][attr].append(y_test_hat)
            mondrian_x['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian_x['low'][attr].append(values[:,0])
            mondrian_x['high'][attr].append(values[:,2])
            mondrian_x['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian_x']['time'] = time() - time_mondrian
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [172]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, crm]
attribute_names = [base, rand, 'crm']
cps = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['median'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['median'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=3)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'crm': crm_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian['uncal'][attr].append(y_test_hat)
            mondrian['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian['low'][attr].append(values[:,0])
            mondrian['high'][attr].append(values[:,2])
            mondrian['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [173]:
# model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, crm]
attribute_names = [base, rand, 'crm']
cps = {}
model_split = {}
model_split['uncal'] = {}
model_split['median'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    cps[attr] = None
    model_split['uncal'][attr] = []
    model_split['median'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=3)
        crm_train_bin = binning(X_train[:,crm], crm_cal_boundaries)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'crm': crm_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'crm': crm_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin,:]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin,:]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin,:]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestRegressor()
                model.fit(X_train_bin, y_train_bin)
                
                residuals_cal = model.predict(X_cal_bin) - y_cal_bin
                y_test_bin_hat = model.predict(X_test_bin)
            
                cps[attr] = ConformalPredictiveSystem()
                cps[attr].fit(residuals_cal)
                values = cps[attr].predict(y_test_bin_hat, lower_percentiles=[5, 50], higher_percentiles=[95, 50])
                model_split['uncal'][attr].append(y_test_bin_hat)
                model_split['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
                model_split['low'][attr].append(values[:,0])
                model_split['high'][attr].append(values[:,2])
                model_split['width'][attr].append(values[:,2]-values[:,0])
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [174]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['median'][attr] = np.concatenate(mondrian_x['median'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['median'][attr] = np.concatenate(mondrian['median'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['median'][attr] = np.concatenate(model_split['median'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [175]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f} ')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 12.4268 	 11.6665 	 11.6932 
rand width: 	 17.0033 	 16.2824 	 22.4043 
crm width: 	 20.2667 	 21.4670 	 21.4383 


In [176]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} MAE: \t{coverage(mondrian_x["y"], mondrian_x["low"][attr], mondrian_x["high"][attr]): .4f} \t{coverage(mondrian["y"], mondrian["low"][attr], mondrian["high"][attr]): .4f} \t{coverage(model_split["y"][attr], model_split["low"][attr], model_split["high"][attr]): .4f}')
for attr in attribute_names:
    print(f'{attr} MAE: \t{mean_absolute_error(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["median"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')
    print(f' \t{mean_absolute_error(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
for attr in attribute_names:
    print(f'{attr} R2: \t{r2_score(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["median"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')    
    print(f' \t{r2_score(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["uncal"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base MAE: 	 0.9391 	 0.9289 	 0.9010
rand MAE: 	 0.9112 	 0.9391 	 0.8909
crm MAE: 	 0.9670 	 0.9543 	 0.9594
base MAE: 	 2.3866 	 2.6365 	 2.4569 	 2.3834 	 2.3444 	 2.4730
rand MAE: 	 2.3831 	 2.6630 	 3.4656 	 2.3834 	 2.3444 	 3.3479
crm MAE: 	 2.4731 	 2.6402 	 2.5975 	 2.3834 	 2.3444 	 2.5554
base R2: 	 0.8380 	 0.8177 	 0.7962 	 0.8388 	 0.8349 	 0.7969
rand R2: 	 0.8394 	 0.8139 	 0.6820 	 0.8388 	 0.8349 	 0.7026
crm R2: 	 0.8344 	 0.8184 	 0.8164 	 0.8388 	 0.8349 	 0.8210


In [177]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


## Communities and Crimes

In [178]:
dataset = 'CaC'
# Load the CSV file
df = pd.read_csv('../data/reg/communities.csv')

# Remove columns with missing values
df = df.dropna(axis=1)

# Remove string columns
df = df.select_dtypes(exclude=['object'])

# df.head()


In [179]:
feature_names = df.columns[:-1]
for i, feature_name in enumerate(feature_names):
    print(f'{i}:{feature_name}', end=", ") 
blk = 4

0:state, 1:fold, 2:population, 3:householdsize, 4:racepctblack, 5:racePctWhite, 6:racePctAsian, 7:racePctHisp, 8:agePct12t21, 9:agePct12t29, 10:agePct16t24, 11:agePct65up, 12:numbUrban, 13:pctUrban, 14:medIncome, 15:pctWWage, 16:pctWFarmSelf, 17:pctWInvInc, 18:pctWSocSec, 19:pctWPubAsst, 20:pctWRetire, 21:medFamInc, 22:perCapInc, 23:whitePerCap, 24:blackPerCap, 25:indianPerCap, 26:AsianPerCap, 27:HispPerCap, 28:NumUnderPov, 29:PctPopUnderPov, 30:PctLess9thGrade, 31:PctNotHSGrad, 32:PctBSorMore, 33:PctUnemployed, 34:PctEmploy, 35:PctEmplManu, 36:PctEmplProfServ, 37:PctOccupManu, 38:PctOccupMgmtProf, 39:MalePctDivorce, 40:MalePctNevMarr, 41:FemalePctDiv, 42:TotalPctDiv, 43:PersPerFam, 44:PctFam2Par, 45:PctKids2Par, 46:PctYoungKids2Par, 47:PctTeen2Par, 48:PctWorkMomYoungKids, 49:PctWorkMom, 50:NumIlleg, 51:PctIlleg, 52:NumImmig, 53:PctImmigRecent, 54:PctImmigRec5, 55:PctImmigRec8, 56:PctImmigRec10, 57:PctRecentImmig, 58:PctRecImmig5, 59:PctRecImmig8, 60:PctRecImmig10, 61:PctSpeakEnglOnly,

In [180]:
target = 'ViolentCrimesPerPop'
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)

In [181]:
Xd, yd = df.drop(target,axis=1), df[target]
X, y = df.drop(target,axis=1).values, df[target].values
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

In [182]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':blk}
attribute_names = [base, rand, 'race']
cps = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['median'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['median'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        x_train, x_cal, x_test = X_train, X_cal, X_test        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(x_train, a, axis=1)
                x_cal = np.delete(x_cal, a, axis=1)
                x_test = np.delete(x_test, a, axis=1)
        model = RandomForestRegressor()
        model.fit(x_train,y_train)
        
        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=5)
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': blk_test_bin}
        residuals_cal = model.predict(x_cal) - y_cal
        y_test_hat = model.predict(x_test)
        mondrian_x['y'].append(y_test)        
        
        for attr, a in attributes.items():                
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian_x['uncal'][attr].append(y_test_hat)
            mondrian_x['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian_x['low'][attr].append(values[:,0])
            mondrian_x['high'][attr].append(values[:,2])
            mondrian_x['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian_x']['time'] = time() - time_mondrian
results[dataset]['mondrian_x']['struct'] = mondrian_x

In [183]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, blk]
attribute_names = [base, rand, 'race']
cps = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['median'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['median'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)

        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=5)
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': blk_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian['uncal'][attr].append(y_test_hat)
            mondrian['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian['low'][attr].append(values[:,0])
            mondrian['high'][attr].append(values[:,2])
            mondrian['width'][attr].append(values[:,2]-values[:,0])

results[dataset]['mondrian']['time'] = time() - time_mondrian
results[dataset]['mondrian']['struct'] = mondrian

In [184]:
# model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, blk]
attribute_names = [base, rand, 'race']
cps = {}
model_split = {}
model_split['uncal'] = {}
model_split['median'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    cps[attr] = None
    model_split['uncal'][attr] = []
    model_split['median'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        
        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=5)
        blk_train_bin = binning(X_train[:,blk], blk_cal_boundaries)  
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)  
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': blk_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': blk_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin,:]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin,:]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin,:]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestRegressor()

                model.fit(X_train_bin, y_train_bin)
                
                residuals_cal = model.predict(X_cal_bin) - y_cal_bin
                y_test_bin_hat = model.predict(X_test_bin)
            
                cps[attr] = ConformalPredictiveSystem()
                cps[attr].fit(residuals_cal)
                values = cps[attr].predict(y_test_bin_hat, lower_percentiles=[5, 50], higher_percentiles=[95, 50])
                model_split['uncal'][attr].append(y_test_bin_hat)
                model_split['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
                model_split['low'][attr].append(values[:,0])
                model_split['high'][attr].append(values[:,2])
                model_split['width'][attr].append(values[:,2]-values[:,0])
                model_split['y'][attr].append(y_test_bin)

results[dataset]['model_split']['time'] = time() - time_split
results[dataset]['model_split']['struct'] = model_split

In [185]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['median'][attr] = np.concatenate(mondrian_x['median'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['median'][attr] = np.concatenate(mondrian['median'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['median'][attr] = np.concatenate(model_split['median'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [186]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results[dataset]['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results[dataset]['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results[dataset]['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f} ')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.4778 	 0.4597 	 0.4360 
rand width: 	 0.5128 	 0.4975 	 0.5145 
race width: 	 0.4511 	 0.4395 	 0.4235 


In [187]:
from sklearn.metrics import mean_absolute_error, r2_score

print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} MAE: \t{coverage(mondrian_x["y"], mondrian_x["low"][attr], mondrian_x["high"][attr]): .4f} \t{coverage(mondrian["y"], mondrian["low"][attr], mondrian["high"][attr]): .4f} \t{coverage(model_split["y"][attr], model_split["low"][attr], model_split["high"][attr]): .4f}')
for attr in attribute_names:
    print(f'{attr} MAE: \t{mean_absolute_error(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["median"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')
    print(f' \t{mean_absolute_error(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
for attr in attribute_names:
    print(f'{attr} R2: \t{r2_score(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["median"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')    
    print(f' \t{r2_score(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["uncal"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base MAE: 	 0.9052 	 0.8967 	 0.8882
rand MAE: 	 0.9168 	 0.9082 	 0.8887
race MAE: 	 0.9142 	 0.9007 	 0.8806
base MAE: 	 0.0989 	 0.1032 	 0.1026 	 0.0942 	 0.0949 	 0.0943
rand MAE: 	 0.0988 	 0.1032 	 0.1107 	 0.0942 	 0.0949 	 0.1018
race MAE: 	 0.0994 	 0.1041 	 0.1055 	 0.0942 	 0.0949 	 0.0973
base R2: 	 0.6460 	 0.6376 	 0.6364 	 0.6530 	 0.6524 	 0.6523
rand R2: 	 0.6462 	 0.6364 	 0.5837 	 0.6530 	 0.6524 	 0.6041
race R2: 	 0.6434 	 0.6320 	 0.6174 	 0.6530 	 0.6524 	 0.6365


In [188]:
import pickle

with open(f'../evaluation/results_mondrian_{dataset}_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)


## Summary
### Time

In [189]:
categories = ['mondrian_x','mondrian','model_split',]
print('Dataset\t Mondrian_x\t Mondrian\t Model Split\tType')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for category in categories:
        print(f'{results[name][category]["time"]: 0.2f}\t\t', end='')
    print(f'{type}')

Dataset	 Mondrian_x	 Mondrian	 Model Split	Type
COMPAS	 12.52		 15.77		 81.94		classification
Adult	 68.95		 65.43		 281.76		classification
German	 4.29		 4.37		 37.07		classification
Housing	 4.87		 5.00		 24.26		regression
CaC	 139.48		 135.64		 399.27		regression


### Width
#### Mondrian vs model split

In [190]:
print('', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in categories:
        print(f' {category}', end='\t')
print('\nDataset', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in categories:
        print(f' {attr}', end='\t\t')
print('Type')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for attr in results[name]['mondrian'].keys():
        if attr in ['time', 'struct']:
            continue
        for category in categories:
            val = results[name][category][attr]
            print(f'{val: 0.3f}', end='\t\t') if val > 0 else print(' -', end='\t\t')
    print(f'{type}')

	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	
Dataset	 base		 base		 base		 rand		 rand		 rand		 race		 race		 race		 sex		 sex		 sex		 age		 age		 age		 edu		 edu		 edu		 crm		 crm		 crm		Type
COMPAS	 0.008		 0.007		 0.007		 0.023		 0.022		 0.020		 0.019		 0.019		 0.019		 0.012		 0.011		 0.011		 0.025		 0.021		 0.020		 -		 -		 -		 -		 -		 -		classification
Adult	 0.003		 0.003		 0.003		 0.009		 0.010		 0.010		 0.006		 0.007		 0.007		 0.004		 0.005		 0.005		 -		 -		 -		 0.015		 0.016		 0.015		 -		 -		 -		classification
German	 0.037		 0.047		 0.042		 0.119		 0.135		 0.118		 -		 -		 -		 0.096		 0.101		 0.089		 0.120		 0.138		 0.126		 -		 -		 -		 -		 -		 -		classification
Housing	 12.427		 11.667		 11.693		 17.003		 16.282		 22.404		 -		 -		 -		 -		 -		 -		 -		 -		 -		 -		 -		 -		 20.267		 

#### Only Mondrian

In [191]:
print('', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian']:
        print(f' {category}', end='\t')
print('\nDataset', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian']:
        print(f' {attr}', end='\t\t')
print('Type')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for attr in results[name]['mondrian'].keys():
        if attr in ['time', 'struct']:
            continue
        for category in ['mondrian']:
            val = results[name][category][attr]
            print(f'{val: 0.3f}', end='\t\t') if val > 0 else print(' -', end='\t\t')
    print(f'{type}')

	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	
Dataset	 base		 rand		 race		 sex		 age		 edu		 crm		Type
COMPAS	 0.007		 0.022		 0.019		 0.011		 0.021		 -		 -		classification
Adult	 0.003		 0.010		 0.007		 0.005		 -		 0.016		 -		classification
German	 0.047		 0.135		 -		 0.101		 0.138		 -		 -		classification
Housing	 11.667		 16.282		 -		 -		 -		 -		 21.467		regression
CaC	 0.460		 0.498		 0.440		 -		 -		 -		 -		regression


In [192]:
import pickle

with open(f'../evaluation/results_mondrian_{str(num_rep)}.pkl', 'wb') as file:
    pickle.dump(results, file)
